In [55]:
%%writefile weatherpy_ehendricks.py

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Save latitudes for use in analysis
lats = []

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)
        lats.append(lat_lng[0])
        
# Print the city count to confirm sufficient count
len(cities)

# Output file
cities_df = pd.DataFrame(cities, columns=["Cities"])
cities_df.to_csv(output_data_file, index=False)

## Perform API Calls

# OpenWeatherMap API Key
api_key = api_keys.api_key

# Starting URL for Weather Map API Call
base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key 

# Setup lists for data
temp = []
humidity = []
clouds = []
wind_speed = []
record_num = 0
error_count = 0

# TEST TEST TEST TEST
#test_cities = cities[:50]
#test_lats = lats[:50]
# TEST TEST TEST TEST

# Loop through the cities
print("Beginning Data Retrieval")
print("------------------------")

# Loop through cities
for city in cities:
    city_query = "&q=" + city
    url = base_url + city_query
    print(f'Processing Record {record_num} of Set 1 | {city}')
    print(url)
    
    # increment
    record_num += 1
    
    try:
        # Get data from OpenWeatherMap
        weather_data = requests.get(url).json()
    
        # Save data
        temp.append(weather_data["main"]["temp_max"])
        humidity.append(weather_data["main"]["humidity"])
        clouds.append(weather_data["clouds"]["all"])
        wind_speed.append(weather_data["wind"]["speed"])

    except KeyError:
        print(f'\nCITY NOT FOUND, skipping {city}\n')
        
        # Keep track of total number of errors
        error_count += 1
                
        # Nullify not found city rows
        temp.append(np.nan)
        humidity.append(np.nan)
        clouds.append(np.nan)
        wind_speed.append(np.nan)

    
print("------------------------")
print("Ending Data Retrieval")
print(f'\nTotal Errors: {error_count}')

data_dict = {"City": cities, "Lat": lats, "Temp": temp, "Humidity": humidity, "Clouds": clouds, "Wind_Speed": wind_speed}
city_weather_df = pd.DataFrame(data_dict)

# Drop the not found cities
city_weather_df.dropna(inplace=True)

city_weather_df.head()

# Scatter Plots
# Temperature (F) vs. Latitude
x_axis = city_weather_df['Lat']
data_to_plot = city_weather_df['Temp']

plt.scatter(x_axis, data_to_plot, marker="o", c='b', edgecolors="black", alpha=0.75, label="Temp (F)")

# Save Figure
plt.savefig("temp_vs_Latitude_plot.png", bbox_inches="tight")

# Incorporate the other graph properties
plt.ylim(0, 120)
plt.xlim(-80, 100)
plt.grid(axis='both', alpha=0.5)
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (F)")
plt.title("City Latitude vs Max Temperature (8/5/2018)")

# Show plot
plt.show()

# Scatter Plots
# Humidity vs. Latitude
x_axis = city_weather_df['Lat']
data_to_plot = city_weather_df['Humidity']

plt.scatter(x_axis, data_to_plot, marker="o", c='b', edgecolors="black", alpha=0.75, label="Temp (F)")

# Save Figure
plt.savefig("humidity_vs_Latitude_plot.png", bbox_inches="tight")

# Incorporate the other graph properties
plt.ylim(0, 120)
plt.xlim(-80, 100)
plt.grid(axis='both', alpha=0.5)
plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.title("City Latitude vs Humidity (8/5/2018)")


# Show plot
plt.show()

# Scatter Plots
# Cloudiness vs. Latitude
x_axis = city_weather_df['Lat']
data_to_plot = city_weather_df['Clouds']

plt.scatter(x_axis, data_to_plot, marker="o", c='b', edgecolors="black", alpha=0.75, label="Temp (F)")

# Save Figure
plt.savefig("clouds_vs_Latitude_plot.png", bbox_inches="tight")

# Incorporate the other graph properties
plt.ylim(-20, 120)
plt.xlim(-80, 100)
plt.grid(axis='both', alpha=0.5)
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.title("City Latitude vs Cloudiness (8/5/2018)")


# Show plot
plt.show()

# Scatter Plots
# Wind Speed vs. Latitude
x_axis = city_weather_df['Lat']
data_to_plot = city_weather_df['Wind_Speed']

plt.scatter(x_axis, data_to_plot, marker="o", c='b', edgecolors="black", alpha=0.75, label="Temp (F)")

# Save Figure
plt.savefig("wind_speed_vs_Latitude_plot.png", bbox_inches="tight")

# Incorporate the other graph properties
plt.ylim(-5, 40)
plt.xlim(-80, 100)
plt.grid(axis='both', alpha=0.5)
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")
plt.title("City Latitude vs Wind Speed (8/5/2018)")


# Show plot
plt.show()

# Analysis


#### Temperature:
There is (generally) an arch to the data.  However, the peak is around 30° (north). 23.5S and 23.5N is considered the tropics so that makes sense as far as north of the equator. Not so much south. That may be due to lack of cities.  But the majority in the southern region of the tropics is less than in the northern part.

The normal temps for the poles in the summer is 32F (north) and -18F (south). In the winter it's -40F (north) and -76F (south). The temps for the north pole (+90°) seems pretty close to the summer normal, except for one plot point being at around 90F. That needs to be verified. The south pole is in winter and the temps are not even close to the normal for winter there.  But there are no points at -90° which makes sense since there are no cities in Antarctica. The most southern city is at -55°, so those plot points to the left of that latitude should be verified. In addition, there are two points that seem cold for the equator (around 40° and 50°). That may be due to the city being in mountains, purhaps.

#### Humidity:
First there are 3 plot points beyond -55° where there are not supposed to be any cities. And there are several plot points near 0° that are low humidity where I would expect high humidity (tropics).  Again, that could be due to altitude.

There is a general bowl shape to the data. I would actually would have expected the bowl to be upside down here, given that the tropics is usually humid jungle (I believe). There are several points between 80-90° that range from 100% to 35%(ish) humidity. But maybe that is due places being cooler? (like side of ice burg?) I know well how warmer air can hold more mositure than cold air because of living most of my life in St. Louis. Anyway, those points should be verified.

#### Cloudiness
"Clouds form when humid air cools enough for water vapor to condense into droplets or ice crystals. The altitude at which this happens depends on the humidity and the rate at which temperature drops with elevation." They also start forming around 12.5 miles above the ground. I don't see how the humidity and temp plots can explain the cloudiness plot. I expect there are other factors involved in cloudiness. How rapidly the air cools as it rises. Another factor in cloud formation are how much (or if) dust, soot, and air pollution is present.

#### Wind Speed:
Wind is created by a difference in pressure going from high to low. Pressure is created by differences in heating the earth. Most of the plot points are below about 12mph (?). There are more above that line in the northern hemisphere and that make sense since its summer in the northern hemisphere.

#### Final Comments:
I already stated that the most southern city on the planet is at -55°. Looking at the closest cities to the north pole, the inhabited ones are at least 20°+ south of the north pole. So anything more than 70° latitude should be verified along with any points less than -55° latitude.

I found it difficult to come up with any conclusions because of the several other factors involved in each of these measurements. Hence the quoting of basic weather stuff. It would be interesting to get historical data and see how these measurements would be different due to climate change. Comparing the tempurature plot to the one in the pdf, it's noticably more scattered (a lot at higher temps) in the current plot.

Overwriting weatherpy_ehendricks.py
